In [1]:
#Importing the necessary packages for the workspace and experiment.
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-122882")
exp = Experiment(workspace=ws, name="AUTO_ML")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-122882
Azure region: southcentralus
Subscription id: b4a122b5-b4d5-40e7-9878-57b87adf4a8b
Resource group: aml-quickstarts-122882


In [3]:
#Importing the necessary packages for computetarget, amlcompute and computetargetexception.
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
cluster_name="AUTOM-ML-cluster"
try:
  compute_target=ComputeTarget(workspace=ws,name=cluster_name)
  print("Found existing compute target")
except ComputeTargetException:
  print('Creating a new compute target')
  compute_config=AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',max_nodes=4)
  compute_target=ComputeTarget.create(ws,cluster_name,compute_config)
  

compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

### YOUR CODE HERE ###

Creating a new compute target
AUTOM-ML-cluster AmlCompute Creating
prone ComputeInstance Succeeded
cpu-cluster-2 AmlCompute Succeeded
cl AmlCompute Succeeded


In [4]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data_path)

In [7]:
def clean_data(data):
    # Dict for cleaning data
    months = {"jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6, "jul":7, "aug":8, "sep":9, "oct":10, "nov":11, "dec":12}
    weekdays = {"mon":1, "tue":2, "wed":3, "thu":4, "fri":5, "sat":6, "sun":7}

    # Clean and one hot encode data
    x_df = data.to_pandas_dataframe().dropna()
    jobs = pd.get_dummies(x_df.job, prefix="job")
    x_df.drop("job", inplace=True, axis=1)
    x_df = x_df.join(jobs)
    x_df["marital"] = x_df.marital.apply(lambda s: 1 if s == "married" else 0)
    x_df["default"] = x_df.default.apply(lambda s: 1 if s == "yes" else 0)
    x_df["housing"] = x_df.housing.apply(lambda s: 1 if s == "yes" else 0)
    x_df["loan"] = x_df.loan.apply(lambda s: 1 if s == "yes" else 0)
    contact = pd.get_dummies(x_df.contact, prefix="contact")
    x_df.drop("contact", inplace=True, axis=1)
    x_df = x_df.join(contact)
    education = pd.get_dummies(x_df.education, prefix="education")
    x_df.drop("education", inplace=True, axis=1)
    x_df = x_df.join(education)
    x_df["month"] = x_df.month.map(months)
    x_df["day_of_week"] = x_df.day_of_week.map(weekdays)
    x_df["poutcome"] = x_df.poutcome.apply(lambda s: 1 if s == "success" else 0)

    y_df = x_df.pop("y").apply(lambda s: 1 if s == "yes" else 0)
    return x_df,y_df

import pandas as pd
# Using the same clean data function to clean the imported csv file dataset.
x, y = clean_data(ds)
print(type(x))
print(type(y))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [8]:
#converting the output to predict-y to dataframe
y_dataframe=pd.DataFrame(y,columns=['y'])
y_dataframe.head(5)

,y
0,0
1,0
2,0
3,0
4,0


In [19]:
#Concatenating the converted dataframe for the output y to the input columns.
converted_data=pd.concat([x,y_dataframe],axis=1)

In [20]:
#Saving the dataframe to a csv file.
converted_data.to_csv('training/full_data.csv')

In [21]:
datastore=ws.get_default_datastore()
datastore

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-1ee6bcf7-2ab0-4383-8511-fe01a2fc2176",
  "account_name": "mlstrg122882",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [22]:
datastore.upload(src_dir='training/',target_path='d/')

Uploading an estimated of 2 files
Target already exists. Skipping upload for d/train_data.csv
Uploading training/full_data.csv
Uploaded training/full_data.csv, 1 files out of an estimated total of 2
Uploaded 1 files


$AZUREML_DATAREFERENCE_0300517ffee04a1eb3d11d63da36b8e6

In [14]:
from azureml.core import Dataset
full_data=Dataset.Tabular.from_delimited_files(path=[(datastore,('d/full_data.csv'))])

In [15]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=full_data,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=compute_target,
    iterations=100,
    max_concurrent_iterations=8)

In [16]:
automl_run=exp.submit(automl_config, show_output=True)


Running on remote.
Running on remote compute: AUTOM-ML-cluster
Parent Run ID: AutoML_9efeb3f9-af4d-4f2f-9d48-095a220cb621

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Beginning model selection.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+----------------

In [17]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [18]:
automl_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|3692                             |1                                |32950                                 |
+---------------------------------+----------

{'runId': 'AutoML_9efeb3f9-af4d-4f2f-9d48-095a220cb621',
 'target': 'AUTOM-ML-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-10-25T21:54:13.802538Z',
 'endTimeUtc': '2020-10-25T22:39:26.225817Z',
 'properties': {'num_iterations': '100',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'AUTOM-ML-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"AUTO_ML","subscription_id":"b4a122b5-b4d5-40e7-9878-57b87adf4a8b","resource_group":"aml-quickstarts-122882","workspace_name":"quick-starts-ws-122882","region":"southcentralus","compute_target":"AUTOM-ML-cluster","spark_service":null,"azure_service":"remote","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":100,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":nul

In [23]:
#Getting the best model output
best_run,model=automl_run.get_output()


In [24]:
#Exporting most optimal model obtained from AutoML using joblib
import joblib
joblib.dump(model,'optimal_model.joblib')

['optimal_model.joblib']

In [25]:
#Printing the best estimator for prediction from AutoML
print(model._final_estimator)


PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('79',
                                           Pipeline(memory=None,
                                                    steps=[('standardscalerwrapper',
                                                            <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7f8778c4ac18>),
                                                           ('xgboostclassifier',
                                                            XGBoostClassifier(base_score=0.5,
                                                                              booster='gbtree',
                                                                              colsample_bylevel=1,
                                                                              colsample_bynode=1,
                                                                              colsample_bytree=1,
                     